In [5]:
!pip install nest_asyncio
import nest_asyncio

# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep
import random
import datetime
from tqdm import tqdm
import re
import random as rand

#Librería para ejecutar de forma asíncerona
import asyncio

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [6]:
def obtener_tabla(texto_tabla, encabezados, localidad):
    """
    Convierte un texto de una tabla en un DataFrame organizado.

    Args:
        texto_tabla (str): Texto crudo que contiene la tabla, separada por saltos de línea.
        encabezados (list): Lista con los encabezados de columna para el DataFrame.
        localidad (str): El nombre de la localidad o municipio al que corresponde la tabla.

    Returns:
        pd.DataFrame: Un DataFrame que contiene los datos organizados con las columnas correctas,
                      incluyendo la localidad y el mes al que corresponde la tabla.
    """
    
    lista_tabla = texto_tabla.split("\n")
    indice_max_avg_min = lista_tabla.index('Max Avg Min')
    lugar_de_division = indice_max_avg_min - 1
    tabla_final = []
    for i in range(1, lugar_de_division+1):
        tabla_final.append(lista_tabla[i::lugar_de_division])

    df_ini = pd.DataFrame(tabla_final)
    df_final = pd.DataFrame()
    for j in range(df_ini.shape[1]):
        df_inter = df_ini[j].str.split(" ", expand=True)
        df_final = pd.concat([df_final, df_inter], axis=1)

    df_final.columns = df_final.iloc[0]

    mes = df_final.iloc[0,0]
    nuevas_columnas = [encabezados[0]]

    for titulo in encabezados[1:-1]:
        for i in range(1,4):
            patron_regex = r"\(.*?\)"  # Buscamos cualquier texto que esté entre paréntesis, incluyendo los paréntesis.
            nuevas_columnas.append(re.sub(patron_regex, " ", titulo) + df_final.columns[i])

    nuevas_columnas.append("Precipitaciones")

    df_final.columns = nuevas_columnas

    df_final.drop(index=0, inplace=True)
    df_final.reset_index(drop=True, inplace=True)
    df_final.insert(column="Mes", loc=0, value=mes)
    df_final.insert(column="Municipio", loc=0, value=localidad)
    
    return df_final



In [7]:
def esperar_y_hacer_click(drivers, ruta_xpath):
    """
    Espera hasta que un elemento esté presente en la página y realiza un clic en él.

    Args:
        drivers (WebDriver): El objeto del navegador (WebDriver) que está siendo utilizado para controlar la página.
        ruta_xpath (str): El XPath del elemento que se está buscando en la página.

    Returns:
        WebElement: El botón o elemento sobre el cual se hizo clic. Retorna `None` si el elemento no se encuentra después de 5 intentos.
    """
    
    boton = None
    intentos = 0
    while True and intentos <= 5:
        try:
            boton = WebDriverWait(drivers, 10).until(EC.presence_of_element_located(("xpath", ruta_xpath)))
            sleep(2)
            boton.click()
            break
        except Exception as e:
            intentos += 1
            print(f"Intento {intentos}: No se ha podido encontrar el botón. Error: {e}")
            continue
    
    return boton

In [8]:

async def navegar_y_extraer_datos(driver, url, municipio):
    """
    Navega a la URL y extrae los datos de una tabla.

    Args:
        driver (webdriver): Instancia del navegador.
        url (str): URL de la página a visitar.
        localidad (str): Nombre de la localidad o municipio.
    
    Returns:
        pd.DataFrame: DataFrame con los datos extraídos.
    """
    
    url_wunder = "https://www.wunderground.com/history"
    driver.maximize_window()
    driver.get(url_wunder)
    await asyncio.sleep(1)  # Simula un pequeño retraso para la carga de la página
    busqueda = municipio 
    iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("css selector", "#sp_message_iframe_1165301")))
    driver.switch_to.frame(iframe)
    driver.implicitly_wait(20)
    driver.find_element("css selector", "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button").click()

    sleep(1)

    driver.switch_to.default_content()

    cuadro_texto = driver.find_element('xpath', '//*[@id="historySearch"]')
    cuadro_texto.send_keys(busqueda)

    WebDriverWait(driver, 10).until(EC.presence_of_element_located(("css selector", '#historyForm > search-autocomplete > ul')))
    sleep(1)
    cuadro_texto.send_keys(Keys.ENTER)
    esperar_y_hacer_click(driver, '//*[@id="dateSubmit"]')
    sleep(2)
    esperar_y_hacer_click(driver, '//*[@id="inner-content"]/div[2]/div[1]/div[1]/div[1]/div/lib-link-selector/div/div/div/a[3]')
    sleep(2)
    df_mes = pd.DataFrame()
    
    # Espera hasta que la tabla esté disponible
    tabla = WebDriverWait(driver, 10).until(EC.presence_of_element_located(('xpath', '//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table')))

    # Extraemos la tabla
    tabla_html = tabla.get_attribute("innerHTML")
    texto_tabla = tabla.text
    sopa_tabla = BeautifulSoup(tabla_html)
    encabezados = []
    for indice_mes in range(9, -1, -1):
        tabla = WebDriverWait(driver, 10).until(EC.presence_of_element_located(('xpath', '//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table')))
        tabla_html = tabla.get_attribute("innerHTML")
        texto_tabla = tabla.text

        sopa_tabla = BeautifulSoup(tabla_html)

        encabezados = []
        for titulo in sopa_tabla.find("tr").findAll("td"):
            encabezados.append(titulo.text)
        df_municipio = obtener_tabla(texto_tabla, encabezados, municipio)
        df_mes = pd.concat([df_mes, df_municipio])
        if indice_mes == 0:
            break
        esperar_y_hacer_click(driver, '//*[@id="monthSelection"]')
        esperar_y_hacer_click(driver, f'//*[@id="monthSelection"]/option[{indice_mes}]')
        esperar_y_hacer_click(driver, '//*[@id="dateSubmit"]')
    return df_municipio
    


async def main():
    df_concatenado = pd.DataFrame()

    # Configuración del navegador Selenium
    driver = webdriver.Chrome()
    url_wunder = "https://www.wunderground.com/history"
    driver.maximize_window()
    driver.get(url_wunder)
    await asyncio.sleep(1)  # Simula un pequeño retraso para la carga de la página
    df = pd.read_csv("datos/coordenadas_municipios.csv")
    lista_municipios = df["Municipio"].unique()
    for municipio in lista_municipios:
        # Creamos una lista de tareas para navegar y extraer datos
        tareas = [navegar_y_extraer_datos(driver, url_wunder, municipio)]

    # Ejecutar todas las tareas en paralelo
    resultados = await asyncio.gather(*tareas)

    # Concatenar los DataFrames resultantes
    df_concatenado = pd.concat(resultados, ignore_index=True)
    return df_concatenado


# Ejecutar la función principal
if __name__ == "__main__":
    
    nest_asyncio.apply()
    display(asyncio.run(main()))
    
    


C:\Users\Administrador\AppData\Local\Temp\ipykernel_14760\1174038075.py:82: RuntimeWarning: coroutine 'navegar_y_extraer_datos' was never awaited
  tareas = [navegar_y_extraer_datos(driver, url_wunder, municipio)]


Intento 1: No se ha podido encontrar el botón. Error: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=129.0.6668.101); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF65D72B095+29557]
	(No symbol) [0x00007FF65D69FA50]
	(No symbol) [0x00007FF65D55B56A]
	(No symbol) [0x00007FF65D56213E]
	(No symbol) [0x00007FF65D5644B7]
	(No symbol) [0x00007FF65D564570]
	(No symbol) [0x00007FF65D5B17AB]
	(No symbol) [0x00007FF65D5A3416]
	(No symbol) [0x00007FF65D5D718A]
	(No symbol) [0x00007FF65D5A2E86]
	(No symbol) [0x00007FF65D5D73A0]
	(No symbol) [0x00007FF65D5F851C]
	(No symbol) [0x00007FF65D5D6F33]
	(No symbol) [0x00007FF65D5A116F]
	(No symbol) [0x00007FF65D5A22D1]
	GetHandleVerifier [0x00007FF65DA5C96D+3378253]
	GetHandleVerifier [0x00007FF65DAA8497+3688311]
	GetHandleVerifier [0x00007FF65DA9D1CB+3642

,Municipio,Mes,Time,Temperature Max,Temperature Avg,Temperature Min,Dew Point Max,Dew Point Avg,Dew Point Min,Humidity Max,Humidity Avg,Humidity Min,Wind Speed Max,Wind Speed Avg,Wind Speed Min,Pressure Max,Pressure Avg,Pressure Min,Precipitaciones
0,zarzalejo,Jan,1,50,41.0,37,41,37.4,34,100,87.3,62,8,2.7,0,28.1,28.0,28.0,0.00
1,zarzalejo,Jan,2,45,40.3,34,43,38.3,32,100,93.2,81,5,1.6,0,28.1,28.1,28.0,0.00
2,zarzalejo,Jan,3,55,50.4,43,54,48.9,43,100,94.9,88,13,4.5,0,28.0,28.0,27.9,0.00
3,zarzalejo,Jan,4,50,48.0,46,50,47.5,46,100,98.2,93,10,2.4,0,28.0,27.9,27.7,0.00
4,zarzalejo,Jan,5,50,45.7,41,46,37.5,32,93,74.0,54,18,8.7,1,27.8,27.7,27.6,0.00
5,zarzalejo,Jan,6,50,40.9,34,34,31.5,28,93,70.5,50,12,3.5,0,28.0,27.9,27.8,0.00
6,zarzalejo,Jan,7,50,38.2,30,34,29.3,27,93,73.1,46,8,2.9,0,28.1,28.0,28.0,0.00
7,zarzalejo,Jan,8,46,36.3,27,32,28.5,25,93,74.5,53,5,2.1,0,28.1,28.0,27.9,0.00
8,zarzalejo,Jan,9,46,41.1,37,37,35.6,32,87,81.0,71,6,2.2,0,28.0,28.0,27.9,0.00
9,zarzalejo,Jan,10,48,43.5,39,41,39.1,37,93,85.4,76,12,4.9,0,28.0,27.9,27.9,0.00
